### FIRST SIMULATION DATASET ANALYSIS

In this Jupyter Notebook we will analyze the first dataset provided by Dr. Langner.
The Dataset has the following characteristics:
The columns in csv file are
 - n -- number of the particle
size -- diameter (in meters), now it is equal to 1.0 for all objects
 - x,y,z -- initial position (meters) in Dimorphos rotating reference frame (IAU_Dimorphos)
 - vx,vy,vz -- initial velocities (meters/second) in Dimorphos rotating reference frame (IAU_Dimorphos)
 - t0 - initial time, in this case it is a dummy column with 0.0 for all objects
 - st - status; 0--survived; 1--escaped; 2--collision with Didimos; 3-- collision with Dimorphos
 - time_end -- time (days) when particle is removed from simulation (impact or escape time or 3000 for survivors). Note: for performance reasons the    current version of the code checks for escapes only at the end of each integration period, in this case it is 1 day, so for escaped objects the time_end is rounded up to a full day, but if you need better precision I can change it.
 - xfinal,yfinal,zfinal,vxfinal,vyfinal,vzfinal -- final positions of the object, these columns contents depend on the status. If st=0 (survived) it is the final position and velocity in barycentric reference frame (non rotating). If st=1 (escaped) -- the values are 0.0. If st=2 it is the impact position and velocity in Didimos rotating reference frame. And for st=3 it is  the impact position and velocity in Dimorphos rotating reference frame.

The object is considered to have escaped when it reaches the distance of 5*r_hill which is ~300 km. This distance is chosen to be simple and to give a very large margin for objects that are temporarily on escape trajectory to return to the system. If I choose a different definition of what we consider an escaped object, the times of escape will be different.

In this notebook we will concentrate on creating a random forest and test it against the linear models.

In [1]:
import pandas as pd
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Ignora tutti i FutureWarning
warnings.filterwarnings(action='ignore', category=FutureWarning)

dataset_path = '../data/raw/1st_simulation_Langner.csv'

try:
    df = pd.read_csv(dataset_path)
    print("Dataset uploaded succesfully")
except FileNotFoundError:
    print(f"Error: file not found for path: {dataset_path}")
except Exception as e:
    print(f"An error occurred: {e}")

#Check if the DataFrame has been uploaded succesfully

print(df.head())

# Define features (X) and labels/target (y)
features = ['x', 'y', 'z', 'vx', 'vy', 'vz']
target = 'st'

X = df[features]
y = df[target]

# Split the data into training and test set
"""
test size: percentage of data in the test set, the remaining is the training set
stratify = y: ensures that each class is well represented in both sets.
random_state = n: fixed seed so that the experiment can be repeated.
"""

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,
                                                    random_state = 42, stratify = y)

print(f'Training Set dimension: {X_train.shape[0]}')
print(f'Test Set Dimension: {X_test.shape[0]}')

#Initialize the scaler
scaler = StandardScaler()

#Get the median and average value from the training set only
X_train_scaled = scaler.fit_transform(X_train)

#Use the same parameters obtained when scaling on the training set on the test set as well
X_test_scaled = scaler.transform(X_test)

Dataset uploaded succesfully
   n  size          x          y          z        vx        vy        vz  \
0  1   1.0  -4.263844 -83.024745 -10.340867  0.033040 -0.088055 -0.033979   
1  2   1.0   0.933023 -83.051149 -10.585260  0.053433 -0.079447 -0.028861   
2  3   1.0  -9.297767 -81.321275 -14.435677 -0.012998 -0.065771 -0.074197   
3  4   1.0 -10.952091 -81.127219 -14.462322 -0.012467 -0.094318 -0.030801   
4  5   1.0 -15.406269 -82.785294  -5.903712 -0.033877 -0.093496 -0.010530   

    t0  st    time_end     xfinal     yfinal     zfinal   vxfinal   vyfinal  \
0  0.0   1   82.000000   0.000000   0.000000   0.000000  0.000000  0.000000   
1  0.0   1   80.000000   0.000000   0.000000   0.000000  0.000000  0.000000   
2  0.0   3    4.442850 -16.523756  78.020506  19.320183  0.014316 -0.059958   
3  0.0   1  237.000000   0.000000   0.000000   0.000000  0.000000  0.000000   
4  0.0   3  159.380358 -21.288058  65.137333 -33.949623  0.174760  0.003633   

    vzfinal  
0  0.000000  
1  0.

## Moving to a Non-Linear Model: Random Forest
After proving that linear models are insufficient, we now move to a Random Forest. This is an ensemble learning method that operates by constructing a multitude of decision trees at training time.

A Random Forest works in two key steps:

- Training (Bagging): It builds hundreds of individual decision trees. Each tree is trained on a random subset of the training data (a "bootstrap" sample). Furthermore, at each split in a tree, it only considers a random subset of the features. This process ensures that the trees are all different and "uncorrelated."

- Prediction (Voting): To classify a new particle, every single tree in the forest makes its own prediction (a "vote"). The Random Forest model then outputs the class that received the majority of the votes.

This approach makes the model extremely robust. By averaging the predictions of many diverse trees, it avoids the "overfitting" that a single decision tree might suffer from. Most importantly for our problem, a Random Forest is highly non-linear; it does not try to find a straight line but can create complex, high-dimensional "decision boundaries" to separate the classes.

In [3]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter

#Check the original distribution of the dataset
print(f'Original training set distribution: {Counter(y_train)}')

#Initialize SMOTE to oversample the minority classes
#Instead of the standard value 5, we set k_neighbors to 7, which is still fine since the class 0 has 8 data points
smote_sampler = SMOTE(random_state=42, k_neighbors=7, n_jobs=1)

#Create the synthetic training data set
X_train_smote, y_train_smote = smote_sampler.fit_resample(X_train_scaled, y_train)

#Check the new distribution
print(f'Current distribution of the dataset, after applying the SMOTE oversampling: {Counter(y_train_smote)}')



Original training set distribution: Counter({3: 397, 2: 227, 1: 168, 0: 8})
Current distribution of the dataset, after applying the SMOTE oversampling: Counter({3: 397, 1: 397, 2: 397, 0: 397})


In [6]:
#Initialize Random Forest
""" 
n_estimators = 100 is a good starting point, we use 100 trees.
n_jobs = -1: use all the processors to parallelize the problem.
"""

n_estimators = 200

rf_model = RandomForestClassifier(n_estimators,
                                  random_state=42,
                                  n_jobs=-1)


# Train the model
print("Started Random Forest training...")
rf_model.fit(X_train_smote, y_train_smote)
print("Training Completed.")

#Use the model to predict the test set
y_pred_rf = rf_model.predict(X_test_scaled)

#Print the classification report
print(f"Report Random Forest (w\ SMOTE) and {n_estimators} trees")
print(classification_report(y_test, y_pred_rf, zero_division=0))


Started Random Forest training...
Training Completed.
Report Random Forest (w\ SMOTE) and 200 trees
              precision    recall  f1-score   support

           0       0.11      0.50      0.18         2
           1       0.25      0.24      0.24        42
           2       0.36      0.46      0.40        57
           3       0.60      0.47      0.53        99

    accuracy                           0.42       200
   macro avg       0.33      0.42      0.34       200
weighted avg       0.45      0.42      0.43       200



We trained the RandomForestClassifier on the exact same SMOTE data that failed to produce results with Logistic Regression. The improvement was immediate and significant. The Random Forest achieved a weighted avg F1-score of 0.45, a substantial improvement over the 0.37 from our best linear model. Crucially, the F1-score for the rare class 0 jumped from 0.00 to 0.17. An analysis of the recall (0.50) shows this means the model successfully identified one of the two st=0 samples in the test set for the first time. This confirms our hypothesis: the data contains a valid, predictable signal, but it is non-linear. The Random Forest's ability to create complex decision boundaries allowed it to find the st=0 cluster, which the linear model was geometrically incapable of separating. This 0.45 F1-score now serves as our new, non-linear performance baseline.